# Azure AI Agent service - Bing integration

<img src="https://learn.microsoft.com/en-us/azure/ai-services/agents/media/agent-service-the-glue.png" width=800>

> https://learn.microsoft.com/en-us/azure/ai-services/agents/

In [1]:
import datetime
import os
import sys
import time

from azure.ai.agents import AgentsClient
from azure.ai.agents.models import BingGroundingTool, MessageRole, ListSortOrder
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv

In [2]:
load_dotenv("azure.env")

True

In [3]:
print(f"Today is {datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')}")

Today is 16-Jun-2025 15:04:40


## Grounding Bing Search

In [4]:
project_client = AgentsClient(
    endpoint=os.getenv("PROJECT_ENDPOINT"),
    credential=DefaultAzureCredential(),
)

### Create connection to bing grounding service

In [5]:
# Shoud be: /subscriptions/<subscription_id>/resourceGroups/<resource_group_name>/providers/Microsoft.CognitiveServices/accounts/<ai_service_name>/projects/<project_name>/con#nections/<connection_name>
BING_CONN_ID = os.getenv("BING_CONN_ID")

In [6]:
bing = BingGroundingTool(connection_id=BING_CONN_ID)
bing

### Create agent with Bing Grounding tool

In [7]:
# Create the agent with the Bing grounding tool
agent = project_client.create_agent(
    model="gpt-4o",
    temperature=0.7,
    name="Bing-grounding-agent-demo",
    instructions="You are an AI Agent that can do some web search using Bing",
    tools=bing.definitions,
    headers={"x-ms-enable-preview": "true"},
    description="Agent with Grounding Bing Search",
    )

print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_Bec0Qgf94V0aViRqtLNvfT6F


### Example 1

In [8]:
prompt = "What is the current price of the MICROSOFT stock in USD?"

In [9]:
# Step 1: Create a thread
thread = project_client.threads.create()
print(f"🧵 [1] Created thread | ID: {thread.id}")

# Step 2: Send the user's message
message = project_client.messages.create(thread_id=thread.id,
                                         role="user",
                                         content=prompt)
print(f"💬 [2] Created message | ID: {message.id}")

# Step 3: Start a run
run = project_client.runs.create(thread_id=thread.id,
                                 agent_id=agent.id)
print(f"🏃 [3] Created run | ID: {run.id}")

# Step 4: Poll run status
print("⏳ [4] Polling run status...")
while run.status in {"queued", "in_progress", "requires_action"}:
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"🔄 Status: {run.status}")

    if run.status == "failed":
        print(f"❌ Run failed | Error: {run.last_error}")
        break

# Step 5: Display agent responses
print("\n🤖 [5] Agent responses:")
messages = project_client.messages.list(thread_id=thread.id,
                                        order=ListSortOrder.ASCENDING)

for msg in messages:
    print("\033[1;34m")  # Blue text for agent messages
    if msg.text_messages:
        for text_message in msg.text_messages:
            print(f"💡 → {text_message.text.value}")
    if msg.url_citation_annotations:
        print("\033[1;32m")  # Green text for citations
        for annotation in msg.url_citation_annotations:
            print(
                f"🔗 URL Citation: [{annotation.url_citation.title}]({annotation.url_citation.url})"
            )

print("\033[0m")  # Reset color

🧵 [1] Created thread | ID: thread_o14Kw5RCKTo3xkowztwm481J
💬 [2] Created message | ID: msg_gp2m9khkamUrKz7ikhUjwhJY
🏃 [3] Created run | ID: run_CndzWjTbm3JCKsllb4yOodUx
⏳ [4] Polling run status...
🔄 Status: in_progress
🔄 Status: completed

🤖 [5] Agent responses:

💡 → What is the current price of the MICROSOFT stock in USD?

💡 → The current price of Microsoft (MSFT) stock is $474.96 USD, as of the latest market close on June 13, 2025【3:1†source】【3:3†source】.

🔗 URL Citation: [Microsoft (MSFT) Stock Price History & Chart Since 1986](https://wallstreetnumbers.com/stocks/msft/price)
🔗 URL Citation: [Microsoft (MSFT) Stock Price & Overview - Stock Analysis](https://stockanalysis.com/stocks/msft/)



### Example 2

In [11]:
prompt = "What are the news about the 2025 WWDC event?"

In [12]:
# Step 1: Create a thread
thread = project_client.threads.create()
print(f"🧵 [1] Created thread | ID: {thread.id}")

# Step 2: Send the user's message
message = project_client.messages.create(thread_id=thread.id,
                                         role="user",
                                         content=prompt)
print(f"💬 [2] Created message | ID: {message.id}")

# Step 3: Start a run
run = project_client.runs.create(thread_id=thread.id,
                                 agent_id=agent.id)
print(f"🏃 [3] Created run | ID: {run.id}")

# Step 4: Poll run status
print("⏳ [4] Polling run status...")
while run.status in {"queued", "in_progress", "requires_action"}:
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"🔄 Status: {run.status}")

    if run.status == "failed":
        print(f"❌ Run failed | Error: {run.last_error}")
        break

# Step 5: Display agent responses
print("\n🤖 [5] Agent responses:")
messages = project_client.messages.list(thread_id=thread.id,
                                        order=ListSortOrder.ASCENDING)

for msg in messages:
    print("\033[1;34m")  # Blue text for agent messages
    if msg.text_messages:
        for text_message in msg.text_messages:
            print(f"💡 → {text_message.text.value}")
    if msg.url_citation_annotations:
        print("\033[1;32m")  # Green text for citations
        for annotation in msg.url_citation_annotations:
            print(
                f"🔗 URL Citation: [{annotation.url_citation.title}]({annotation.url_citation.url})"
            )

print("\033[0m")  # Reset color

🧵 [1] Created thread | ID: thread_o6ntZWn8HmEvHUo7Hxn4qE2r
💬 [2] Created message | ID: msg_XzrAw4pwT1knpVLhnNMrom5H
🏃 [3] Created run | ID: run_zB0HBO8VMdp3MZCFGhjLl5j5
⏳ [4] Polling run status...
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: completed

🤖 [5] Agent responses:

💡 → What are the news about the 2025 WWDC event?

💡 → The 2025 Apple Worldwide Developers Conference (WWDC) has unveiled significant updates, including:

1. **iOS 26 and Liquid Glass UI**: Apple introduced a major visual overhaul, "Liquid Glass," for iOS 26, featuring shiny, transparent, and responsive design elements. This marks Apple’s most dramatic design change in over a decade【3:1†source】【3:2†source】.

2. **Unified Naming Scheme**: All Apple platforms, such as macOS, iPadOS, and watchOS, are now adopting year-based names (e.g., iOS 26), aligning with the calendar year【3:1†source】.

3. **Gaming and AI Enhancements**: A dedicated Games app centralizes gaming experiences, while "Ap

### Example 3

In [13]:
prompt = "What are the weather informations for Paris? Show the report and use emojis and be funny"

In [14]:
# Step 1: Create a thread
thread = project_client.threads.create()
print(f"🧵 [1] Created thread | ID: {thread.id}")

# Step 2: Send the user's message
message = project_client.messages.create(thread_id=thread.id,
                                         role="user",
                                         content=prompt)
print(f"💬 [2] Created message | ID: {message.id}")

# Step 3: Start a run
run = project_client.runs.create(thread_id=thread.id,
                                 agent_id=agent.id)
print(f"🏃 [3] Created run | ID: {run.id}")

# Step 4: Poll run status
print("⏳ [4] Polling run status...")
while run.status in {"queued", "in_progress", "requires_action"}:
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"🔄 Status: {run.status}")

    if run.status == "failed":
        print(f"❌ Run failed | Error: {run.last_error}")
        break

# Step 5: Display agent responses
print("\n🤖 [5] Agent responses:")
messages = project_client.messages.list(thread_id=thread.id,
                                        order=ListSortOrder.ASCENDING)

for msg in messages:
    print("\033[1;34m")  # Blue text for agent messages
    if msg.text_messages:
        for text_message in msg.text_messages:
            print(f"💡 → {text_message.text.value}")
    if msg.url_citation_annotations:
        print("\033[1;32m")  # Green text for citations
        for annotation in msg.url_citation_annotations:
            print(
                f"🔗 URL Citation: [{annotation.url_citation.title}]({annotation.url_citation.url})"
            )

print("\033[0m")  # Reset color

🧵 [1] Created thread | ID: thread_jZJbWzoUMYMSYouJzXj2AvSs
💬 [2] Created message | ID: msg_BpRfVtPjwz50UmX2ihMalHDk
🏃 [3] Created run | ID: run_pbbIC2n6EDwKnJfY3YcQvNKq
⏳ [4] Polling run status...
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: completed

🤖 [5] Agent responses:

💡 → What are the weather informations for Paris? Show the report and use emojis and be funny

💡 → Today in Paris, it's a mixed bag of weather tricks! 🎩✨ The temperature is a mild 66°F (19°C) with a low of 59°F (15°C). Expect some clouds, maybe rain, and definitely a sprinkle of Parisian flair. 🌥️☔

Don't forget your umbrella! It might rain, but hey, more reason to cozy up in a café with a croissant! 🥐☕ And if the clouds clear, you'll enjoy those charming cobblestone streets under perfect walking weather. 🌈

Stay dry and Eiffel-tastic! 🗼【3:1†source】

🔗 URL Citation: [Paris weather in June 2025 | Paris 14 day weather](https://www.weather25.com/europe/france/ile-de-france/paris?page=month&month=June)



### Example 4

In [15]:
prompt = "What can you tell about Roland Garros 2025?"

In [16]:
# Step 1: Create a thread
thread = project_client.threads.create()
print(f"🧵 [1] Created thread | ID: {thread.id}")

# Step 2: Send the user's message
message = project_client.messages.create(thread_id=thread.id,
                                         role="user",
                                         content=prompt)
print(f"💬 [2] Created message | ID: {message.id}")

# Step 3: Start a run
run = project_client.runs.create(thread_id=thread.id,
                                 agent_id=agent.id)
print(f"🏃 [3] Created run | ID: {run.id}")

# Step 4: Poll run status
print("⏳ [4] Polling run status...")
while run.status in {"queued", "in_progress", "requires_action"}:
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"🔄 Status: {run.status}")

    if run.status == "failed":
        print(f"❌ Run failed | Error: {run.last_error}")
        break

# Step 5: Display agent responses
print("\n🤖 [5] Agent responses:")
messages = project_client.messages.list(thread_id=thread.id,
                                        order=ListSortOrder.ASCENDING)

for msg in messages:
    print("\033[1;34m")  # Blue text for agent messages
    if msg.text_messages:
        for text_message in msg.text_messages:
            print(f"💡 → {text_message.text.value}")
    if msg.url_citation_annotations:
        print("\033[1;32m")  # Green text for citations
        for annotation in msg.url_citation_annotations:
            print(
                f"🔗 URL Citation: [{annotation.url_citation.title}]({annotation.url_citation.url})"
            )

print("\033[0m")  # Reset color

🧵 [1] Created thread | ID: thread_YzBFOqlRaKiUa9GyuGogLxhK
💬 [2] Created message | ID: msg_rjoeEsQ3izMcqAMCHku8gy6q
🏃 [3] Created run | ID: run_aIylYJCc7UjzUsfFdu0tegTo
⏳ [4] Polling run status...
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: completed

🤖 [5] Agent responses:

💡 → What can you tell about Roland Garros 2025?

💡 → Roland Garros 2025, also known as the French Open, concluded on June 8, 2025. The tournament featured intense matches on the iconic clay courts of Stade Roland Garros in Paris. Key highlights include:

- **Carlos Alcaraz** claimed the men's singles title with a victory over Jannik Sinner in the final, marking a significant moment in his career.
- **Coco Gauff** won her second Grand Slam title in the women's singles after defeating Aryna Sabalenka.
- In doubles, **Marcel Granollers** and **Horacio Zeballos** triumphed in the men's category, while British duo **Alfie Hewett** and **Gordon Reid** secured their sixth consecutive wheelchair doubles title【3:0

## Example 5

In [17]:
prompt = "Who won at the Cannes festival in 2025?"

In [18]:
# Step 1: Create a thread
thread = project_client.threads.create()
print(f"🧵 [1] Created thread | ID: {thread.id}")

# Step 2: Send the user's message
message = project_client.messages.create(thread_id=thread.id,
                                         role="user",
                                         content=prompt)
print(f"💬 [2] Created message | ID: {message.id}")

# Step 3: Start a run
run = project_client.runs.create(thread_id=thread.id,
                                 agent_id=agent.id)
print(f"🏃 [3] Created run | ID: {run.id}")

# Step 4: Poll run status
print("⏳ [4] Polling run status...")
while run.status in {"queued", "in_progress", "requires_action"}:
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"🔄 Status: {run.status}")

    if run.status == "failed":
        print(f"❌ Run failed | Error: {run.last_error}")
        break

# Step 5: Display agent responses
print("\n🤖 [5] Agent responses:")
messages = project_client.messages.list(thread_id=thread.id,
                                        order=ListSortOrder.ASCENDING)

for msg in messages:
    print("\033[1;34m")  # Blue text for agent messages
    if msg.text_messages:
        for text_message in msg.text_messages:
            print(f"💡 → {text_message.text.value}")
    if msg.url_citation_annotations:
        print("\033[1;32m")  # Green text for citations
        for annotation in msg.url_citation_annotations:
            print(
                f"🔗 URL Citation: [{annotation.url_citation.title}]({annotation.url_citation.url})"
            )

print("\033[0m")  # Reset color

🧵 [1] Created thread | ID: thread_mAXXryQA7R7WrXmmRAZ0NgAP
💬 [2] Created message | ID: msg_xC6imzgGEq02bqpq21ezsiUo
🏃 [3] Created run | ID: run_C41uqbQ96OgrGpCLteCFgYXz
⏳ [4] Polling run status...
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: completed

🤖 [5] Agent responses:

💡 → Who won at the Cannes festival in 2025?

💡 → At the 2025 Cannes Film Festival, the Palme d'Or was awarded to Iranian filmmaker Jafar Panahi for his movie *It Was Just an Accident*. Other notable wins included:

- Grand Prix: Joachim Trier's *Sentimental Value*
- Jury Prize (tied): Mascha Schilinski's *Sound of Falling* and Oliver Laxe's *Sirât*
- Best Director: Kleber Mendonça Filho for *The Secret Agent*
- Best Actress: Nadia Melliti in *The Little Sister*
- Best Actor: Wagner Moura in *The Secret Agent*【3:0†source】【3:3†source】.

🔗 URL Citation: [The 78th Festival de Cannes winners' list - Festival de Cannes](https://www.festival-cannes.com/en/press/press-releases/the-78th-festival-de-cannes-winners-

### Post processing

In [19]:
agents_list = [agent]

for agent in agents_list:
    print(
        f"Agent Details:\n"
        f"📛 Name        : {agent.name}\n"
        f"🆔 ID          : {agent.id}\n"
        f"🧠 Model       : {agent.model}\n"
        f"📜 Instructions: {agent.instructions}\n"
    )

Agent Details:
📛 Name        : Bing-grounding-agent-demo
🆔 ID          : asst_Bec0Qgf94V0aViRqtLNvfT6F
🧠 Model       : gpt-4o
📜 Instructions: You are an AI Agent that can do some web search using Bing



In [20]:
#print("Deleting the agents we have created...")
#for agent in agents_list:
    #project_client.delete_agent(agent.id)
    #print(f"Deleted agent, agent ID: {agent.id}")
#print("Done")